In [46]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def sound_speed(P, rho):
    """Calculate sound speed."""
#     epsilon = 1e-6  # Small value to avoid division by zero
#     P = max(P, 0.0)  # Ensure pressure is non-negative
    return np.sqrt(gamma * P / rho)

In [168]:
# def compute_flux(U):
#     """Compute the flux vector F for a single row of conserved variables."""
#     rho, rho_v, E = U  # Extract conserved variables from a single row
#     v = rho_v / rho  # Velocity
#     P = (gamma - 1) * (E - 0.5 * rho * v ** 2)  # Pressure

#     # Compute fluxes for mass, momentum, and energy
#     F = np.zeros(3)  # Flux vector with 3 components
#     F[0] = rho_v  # Mass flux
#     F[1] = rho_v * v + P  # Momentum flux
#     F[2] = (E + P) * v  # Energy flux

#     return F



# def compute_flux(U):
#     """Compute the flux vector F for a single row of conserved variables."""
#     rho, rho_v, E = U  # Extract conserved variables
    
#     # Clamp density and pressure to non-negative values
#     rho = max(rho, 1e-8)  # Prevent zero or negative density
#     v = rho_v / rho  # Velocity
#     P = max((gamma - 1) * (E - 0.5 * rho * v**2), 0.0)  # Clamp pressure

#     if rho <= 0:
#         print(f"Warning: Non-positive density encountered! rho = {rho}")
    
#     v = rho_v / rho  # Velocity
#     P = (gamma - 1) * (E - 0.5 * rho * v**2)  # Pressure

#     if P < 0:
#         print(f"Warning: Negative pressure encountered! P = {P}")

#     F = np.zeros(3)  # Flux vector
#     F[0] = rho_v  # Mass flux
#     F[1] = rho_v * v + P  # Momentum flux
#     F[2] = (E + P) * v  # Energy flux

#     return F




# def compute_flux(U):
#     """Compute the flux vector F for a single row of conserved variables."""
#     rho, rho_v, E = U

#     # Clamp density and pressure to non-negative values
#     rho = max(rho, 1e-8)  # Prevent zero or negative density
#     v = rho_v / rho  # Velocity
#     P = max((gamma - 1) * (E - 0.5 * rho * v**2), 0.0)  # Clamp pressure

#     F = np.zeros(3)  # Flux vector
#     F[0] = rho_v  # Mass flux
#     F[1] = rho_v * v + P  # Momentum flux
#     F[2] = (E + P) * v  # Energy flux

#     return F



# def compute_flux(U):
#     """Compute the flux vector F for a single row of conserved variables."""
#     rho, rho_v, E = U

#     # Clamp density to avoid zero or negative values
#     rho = max(rho, 1e-8)

#     # Calculate velocity
#     v = rho_v / rho

#     # Ensure energy is non-negative and sufficient to avoid negative pressure
#     E_min = 1e-8  # Minimum energy threshold
#     E = max(E, E_min)

#     # Calculate pressure with clamping
#     P = max((gamma - 1) * (E - 0.5 * rho * v**2), 0.0)

#     # Calculate fluxes
#     F = np.zeros(3)
#     F[0] = rho_v  # Mass flux
#     F[1] = rho_v * v + P  # Momentum flux
#     F[2] = (E + P) * v  # Energy flux

#     return F

# def compute_flux(U):
#     """Compute the flux vector F for a single row of conserved variables."""
#     rho, rho_v, E = U

#     # Clamp density to avoid zero or negative values
#     rho = max(rho, 1e-8)

#     # Calculate velocity
#     v = rho_v / rho

#     # Clamp energy to avoid negative values
#     E = max(E, 1e-8)

#     # Calculate pressure with clamping
#     P = max((gamma - 1) * (E - 0.5 * rho * v**2), 0.0)

#     # Calculate fluxes
#     F = np.zeros(3)
#     F[0] = rho_v  # Mass flux
#     F[1] = rho_v * v + P  # Momentum flux
#     F[2] = (E + P) * v  # Energy flux

#     return F


def compute_flux(U):
    """Compute the flux vector F for a single row of conserved variables."""
    rho, rho_v, E = U

    # Ensure non-negative density and energy
    rho = max(rho, 1e-8)
    E = max(E, 1e-8)

    # Compute velocity
    v = rho_v / rho

    # Compute pressure, ensuring it’s non-negative
    P = max((gamma - 1) * (E - 0.5 * rho * v**2), 0.0)

    # Adjust pressure sign based on flow direction
    P_direction = -1 if v < 0 else 1  # Reverse pressure if velocity is negative

    # Compute fluxes
    F = np.zeros(3)
    F[0] = rho_v  # Mass flux
    F[1] = rho_v * v + P_direction * P  # Adjust momentum flux direction
    F[2] = (E + P) * v  # Energy flux

    return F


In [169]:
def HLL_flux(UL, UR):
    """HLL Riemann solver to compute flux at the interface."""
    FL = compute_flux(UL)
    FR = compute_flux(UR)

    # Extract variables from left and right states
    rhoL, vL = UL[0], UL[1] / max(UL[0], 1e-8)
    rhoR, vR = UR[0], UR[1] / max(UR[0], 1e-8)

    # Compute pressures
    PL = max((gamma - 1) * (UL[2] - 0.5 * rhoL * vL ** 2), 0.0)
    PR = max((gamma - 1) * (UR[2] - 0.5 * rhoR * vR ** 2), 0.0)

    # Compute sound speeds
    cL = sound_speed(PL, rhoL)
    cR = sound_speed(PR, rhoR)

    # Compute wave speeds
    alpha_minus = min(vL - cL, vR - cR)
    alpha_plus = max(vL + cL, vR + cR)

    # Ensure fluxes are applied correctly based on wave speeds
    if alpha_minus >= 0:
        return FL  # Use left flux if all waves move to the right
    elif alpha_plus <= 0:
        return FR  # Use right flux if all waves move to the left
    else:
        # Intermediate state: mix fluxes
        return (alpha_plus * FL + alpha_minus * FR -
                alpha_plus * alpha_minus * (UR - UL)) / (alpha_plus - alpha_minus)


In [175]:
def update(U, dt):
    """Update conserved variables using fluxes."""
    U_new = np.copy(U)
    for i in range(1, Nx - 1):
        flux_left = HLL_flux(U[i - 1], U[i])
        flux_right = HLL_flux(U[i], U[i + 1])

        print(f"Cell {i}: flux_left = {flux_left}, flux_right = {flux_right}")

        U_new[i] = U[i] - dt / dx * (flux_right - flux_left)

        # Check for negative energy
        if U_new[i, 2] < 0:
            print(f"Warning: Negative energy encountered at cell {i}.")
            print(f"Old energy: {U[i, 2]}, New energy: {U_new[i, 2]}")
            U_new[i, 2] = 1e-8  # Clamp to small positive value

    return U_new


In [171]:
def plot_state(U, t):
    """Plot density, velocity, and pressure at time t."""
    x = np.linspace(x_min, x_max, Nx)
    rho = U[:, 0]  # Density
    v = U[:, 1] / U[:, 0]  # Velocity
    P = (gamma - 1) * (U[:, 2] - 0.5 * U[:, 0] * v**2)  # Pressure

    plt.figure(figsize=(12, 8))

    # Plot density
    plt.subplot(3, 1, 1)
    plt.plot(x, rho, label='Density (rho)')
    plt.ylabel('Density')
    plt.legend()

    # Plot velocity
    plt.subplot(3, 1, 2)
    plt.plot(x, v, label='Velocity (v)', color='orange')
    plt.ylabel('Velocity')
    plt.legend()

    # Plot pressure
    plt.subplot(3, 1, 3)
    plt.plot(x, P, label='Pressure (P)', color='green')
    plt.ylabel('Pressure')
    plt.xlabel('Position (x)')
    plt.legend()

    plt.suptitle(f'State of Shock Tube at t = {t:.3f}')
    plt.tight_layout()
    plt.show()


In [176]:
# Parameters
gamma = 1.4  # Adiabatic index for ideal gas
Nx = 5  # Number of grid cells
x_min, x_max = 0.0, 1.0  # Domain
dx = (x_max - x_min) / Nx  # Grid spacing

# Time step parameters
CFL = 0.5  # Courant number for stability
t_max = 0.5  # Maximum simulation time

In [177]:
# Initialize arrays for conserved variables: U[rho, rho*v, E]
U = np.zeros((Nx, 3))  # U[i] = [rho_i, rho*v_i, E_i]

# Initial conditions with smaller discontinuity
U[: Nx // 2, 0] = 1.0  # Left side: rho = 1.0
U[Nx // 2 :, 0] = 0.9  # Right side: rho = 0.9 (instead of 0.125)
U[: Nx // 2, 2] = 1.0 / (gamma - 1)  # Internal energy on the left
U[Nx // 2 :, 2] = 0.5 / (gamma - 1)  # Internal energy on the right

In [178]:
# Time integration loop
t = 0.0

while t < t_max:
    # Compute maximum wave speed for CFL condition
    max_speed = 0.0
    for i in range(Nx):
        rho, rho_v, E = U[i]
        v = rho_v / rho
        P = (gamma - 1) * (E - 0.5 * rho * v ** 2)
        c = sound_speed(P, rho)
        max_speed = max(max_speed, abs(v) + c)

    dt = CFL * dx / max_speed  # Time step
    if dt <= 0:
        raise ValueError("Invalid time step: dt must be positive!")
    U = update(U, dt)  # Update conserved variables
    t += dt  # Increment time
    
    # Plot state at specific time intervals 
    # If the current time is close to 0.05, 0.1, or 0.2 seconds (within one time step), create a plot.
    
#     if abs(t - 0.05) < dt or abs(t - 0.1) < dt or abs(t - 0.2) < dt:
#         plot_state(U, t)

print(U)

Cell 1: flux_left = [0. 0. 0.], flux_right = [-0.0591608   0.25       -0.73950997]
Cell 2: flux_left = [-0.0591608   0.25       -0.73950997], flux_right = [0. 0. 0.]
Cell 3: flux_left = [0. 0. 0.], flux_right = [0. 0. 0.]
Cell 1: flux_left = [0.07185016 0.99305742 0.41197866], flux_right = [-0.19788022 -0.5884045  -1.42453339]
Cell 2: flux_left = [-0.19788022 -0.5884045  -1.42453339], flux_right = [ 0.06423382 -0.10157781  0.2181958 ]
Cell 3: flux_left = [ 0.06423382 -0.10157781  0.2181958 ], flux_right = [0. 0. 0.]
Cell 1: flux_left = [-0.1083699   0.29771609 -0.147314  ], flux_right = [ 0.1458326   0.75828643 -0.431772  ]
Cell 2: flux_left = [ 0.1458326   0.75828643 -0.431772  ], flux_right = [0.04932456 0.23660646 0.4696231 ]
Cell 3: flux_left = [0.04932456 0.23660646 0.4696231 ], flux_right = [-0.02922533 -0.49763548 -0.07397142]
Cell 1: flux_left = [-0.10745959  0.20928301  0.07172182], flux_right = [ 0.03067796  0.75891252 -1.23284206]
Cell 2: flux_left = [ 0.03067796  0.75891252

<ipython-input-85-aad4bcd8f899>:5: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(gamma * P / rho)
